In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("D:\Downloads\creditcard.csv.zip")

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def z_score_normalization(x):
    Mu = np.mean(x,axis = 0)
    std = np.std(x,axis = 0)
    x_norm = (x-Mu)/(std)

    return x_norm

Next thing is to handle the imbalance in the data that is in this the data have high no of non-fraudelent cases 
so inorder to solve this imabalance we nee to solve this 
1)undersampling
Reduce the number of non-fraud cases (Class 0) so it matches the number of fraud cases (Class 1).

✏️ Example:
From 284,315 non-frauds → randomly pick only 492 to match the 492 frauds.
So now you have:

492 frauds

492 non-frauds
2)oversampling
Duplicate or synthesize more fraud cases (Class 1) so it matches the number of non-fraud cases.

Two types:
🔁 a. Random Oversampling
Simply duplicate existing fraud samples

Easy but may lead to overfitting

🧠 b. SMOTE (Synthetic Minority Oversampling Technique)
Generates new synthetic fraud samples based on real ones

Creates slightly modified “fake” examples to enrich diversity

In [ ]:
# Going with undersampling

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [ ]:
def compute_cost(x,y,w,b):
    x = np.array(x)
    y = np.array(y)
    m = x.shape[0]
    cost = 0 
    for i in range(m):
        z_i = np.dot(x[i],w)+b
        f_wb_i = sigmoid(z_i)
        cost+=-y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
    cost/=(m)
    return cost
        

In [ ]:
w_init = np.zeros(X_train_balanced.shape[1])
b_init = 0

In [ ]:
def compute_gradient_logistic(x,y,w,b):
    x = np.array(x)
    y = np.array(y)
    m,n = x.shape
    dj_dw = np.zeros((n,))
    dj_db = 0
    for i in range(m):
        f_wb_i = sigmoid(np.dot(x[i],w)+b)
        err_i  = (f_wb_i - y[i])
        for j in range(n):
            dj_dw+=err_i*x[i,j]

        dj_db+=err_i
    dj_dw = dj_dw/(m)
    dj_db = dj_db/(m)
    return dj_dw,dj_db

In [ ]:
import math

In [ ]:
def compute_gradient_descent(x,y,w,b,num_iterations,alpha):

    j_history = []
    for i in range(num_iterations):
        dj_dw,dj_db = compute_gradient_logistic(X_train_balanced,y_train_balanced,w,b)
        w = w - alpha*dj_dw
        b = b - alpha*dj_db
        if i<1000000:
             j_history.append(compute_cost(X_train_balanced,y_train_balanced,w,b))
        if i% math.ceil(num_iterations / 10) == 0:
             print(f"Iteration {i:4d}: Cost {j_history[-1]}")
    return w,b,j_history

In [ ]:
def predict(x,w,b):
    z = np.dot(x,w)+b
    return sigmoid(z)>=0.9

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

new_df = df.copy()

new_df['time_bin'] = pd.cut(new_df['Time'],bins = np.arange(0,new_df['Time'].max()+3600,3600))

new_df = new_df[new_df['time_bin'].notna()]

avg_amount_per_time_bin = new_df.groupby('time_bin')['Amount'].mean()
avg_amount_per_time_bin.name = 'average_amount_per_time_bin'
new_df = new_df.join(avg_amount_per_time_bin,on = 'time_bin')


In [ ]:
new_df = new_df[new_df['average_amount_per_time_bin'].notna()]

In [ ]:
new_df['Amount_by_Avg'] = new_df['Amount']/new_df['average_amount_per_time_bin']
new_df['log_amount_by_avg'] = np.log1p(new_df['Amount_by_Avg'])


In [ ]:

new_df['log_Amount_by_Avg_scaled'] = z_score_normalization(new_df[['log_amount_by_avg']])

In [ ]:
new_df

In [ ]:
new_df.shape

In [ ]:
w_init = np.zeros(X_train_balanced.shape[1])
b_init = 0

In [ ]:

new_df.drop(["average_amount_per_time_bin"],axis=1)


In [ ]:
new_df

In [ ]:
new_df.shape[0]

In [ ]:
norm_amount = z_score_normalization(new_df[['Amount']])
norm_Time   = z_score_normalization(new_df[['Time']])

In [ ]:
new_df = new_df.join([norm_amount.rename(columns={'Amount': 'norm_amount'}),
                      norm_Time.rename(columns={'Time': 'norm_time'})])

In [ ]:
new_df = new_df.drop(['Time'],axis = 1)

In [ ]:
new_df = new_df.drop(['Amount'],axis = 1)

In [ ]:
new_df = new_df.drop(['average_amount_per_time_bin'],axis = 1)

In [ ]:
new_df

In [ ]:
x = new_df.drop('Class',axis = 1)
Y = new_df[['Class']]

In [ ]:
x

In [ ]:
Y

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x,Y,test_size = 0.2,random_state = 42,stratify = Y)

In [ ]:
new_df.columns[new_df.isnull().any()]

In [ ]:
train_data_new = pd.concat([X_train,y_train],axis = 1)

In [ ]:
fraud_case_new = train_data_new[train_data_new["Class"]==1]

In [ ]:
non_fraud_case_new = train_data_new[train_data_new["Class"]==0]

In [ ]:
fraud_case_new

In [ ]:
non_fraud_case_new

In [ ]:
non_fraud_case_sampled = non_fraud_case_new.sample(n=len(fraud_case_new),random_state=42)

In [ ]:
count = 0
for i in range(len(fraud_case_new[fraud_case_new["Class"]==1])):
    count+=1
print(count)

In [ ]:
count = 0
for i in range(len(non_fraud_case_sampled[non_fraud_case_sampled["Class"]==0])):
    count+=1
print(count)

In [ ]:
balanced_data_new = pd.concat([fraud_case_new,non_fraud_case_sampled]) 

In [ ]:
balanced_data_new

In [ ]:
balanced_data_new = balanced_data_new.sample(frac = 1,random_state = 42).reset_index(drop = True)
X_train_balanced_new = balanced_data_new.drop('Class',axis = 1)
y_train_balanced_new = balanced_data_new[['Class']]

In [ ]:
X_train_balanced_new

In [ ]:
X_train_balanced_new=X_train_balanced_new.drop('time_bin',axis = 1)

In [ ]:
X_train_balanced_new=X_train_balanced_new.drop('Amount_by_Avg',axis = 1)

In [ ]:
X_train_balanced_new=X_train_balanced_new.drop('log_amount_by_avg',axis = 1)

In [ ]:
X_train_balanced_new

In [ ]:
y_train_balanced_new

In [ ]:
w_init_new = np.zeros(X_train_balanced_new.shape[1])
b_init_new = 0

In [ ]:
print(X_train_balanced_new.dtypes)

In [ ]:
def compute_gradient_descent(x,y,w,b,num_iterations,alpha):

    j_history = []
    for i in range(num_iterations):
        dj_dw,dj_db = compute_gradient_logistic(X_train_balanced_new,y_train_balanced_new,w,b)
        w = w - alpha*dj_dw
        b = b - alpha*dj_db
        if i<1000000:
             j_history.append(compute_cost(X_train_balanced_new,y_train_balanced_new,w,b))
        if i% math.ceil(num_iterations / 10) == 0:
             print(f"Iteration {i:4d}: Cost {j_history[-1]}")
    return w,b,j_history

In [ ]:
num_iterations = 1000
alpha = 0.01
w,b,j_history = compute_gradient_descent(X_train_balanced_new,y_train_balanced_new,w_init_new,b_init_new,num_iterations,alpha)

In [ ]:
print(w,b)

In [ ]:
X_test

In [ ]:
X_test = X_test.drop(columns = ['time_bin','Amount_by_Avg','log_amount_by_avg'])


In [230]:
X_test

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,log_Amount_by_Avg_scaled,norm_amount,norm_time
21621,1.214370,0.090817,0.677072,0.918274,-0.724679,-0.909902,-0.024860,-0.151567,0.459109,-0.189292,...,-0.431827,0.056888,0.740339,0.407431,0.249798,-0.022020,0.015413,-0.728576,-0.337516,-1.327622
222881,-0.253260,0.675054,0.595508,0.072451,-0.145390,-0.031321,0.943205,-0.256332,0.500910,-0.500424,...,1.510042,-0.307591,0.037256,-0.427784,0.592583,-0.006365,0.181051,0.242416,-0.031424,1.018045
179908,-0.501615,0.401011,0.772183,-0.768753,0.858744,-0.431940,1.440318,-0.307876,-0.490776,-0.474085,...,-0.385504,-0.055339,-0.328323,-0.092642,0.246250,-0.152176,-0.107283,0.190757,-0.033303,0.620572
94783,1.268652,-0.773642,1.099085,-0.484241,-1.583179,-0.321417,-1.037697,0.068260,-0.566151,0.713578,...,0.068506,0.008803,0.618262,0.151121,1.132142,-0.020658,0.010562,-0.425407,-0.281183,-0.627228
192837,-1.284036,1.611890,2.333462,3.886285,-0.415366,2.462808,-0.368639,0.742133,-0.814311,2.436457,...,0.575453,-0.363000,0.066300,0.181497,0.569632,0.788244,0.318371,0.338655,-0.001198,0.738203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171972,2.151208,-0.801270,-1.464095,-1.083325,-0.311628,-0.623375,-0.460369,-0.168283,-0.799255,1.027911,...,1.343875,0.006736,0.763895,0.166010,0.016955,-0.048080,-0.066349,-0.350058,-0.233206,0.549312
208350,-1.328215,1.109017,1.620232,-0.376369,-0.391954,-0.335922,0.239156,0.430883,-0.027075,-0.987547,...,-0.412697,-0.231756,-0.021410,0.384435,-0.409657,0.102640,0.030651,-0.269649,-0.213336,0.890075
138723,1.183463,-0.178754,0.662303,-0.172752,-0.720783,-0.366493,-0.444363,0.156813,0.249733,-0.062468,...,-0.179780,0.150431,0.262954,-0.041665,0.906992,-0.063386,-0.004046,-0.763549,-0.346832,-0.252772
156844,-0.509159,0.857570,2.404118,-0.119611,0.178800,-0.238802,0.600110,-0.292371,1.406421,-1.173361,...,-0.354668,-0.286081,-0.098939,0.276301,-0.727369,-0.075017,-0.144286,-0.469517,-0.308170,0.299330


In [ ]:
def predict_new(x,w,b):
    z = np.dot(x,w)+b
    return sigmoid(z)>=0.9

In [ ]:
y_pred = predict_new(X_test,w,b)

In [231]:
y_pred


array([False, False, False, ..., False, False, False])

In [ ]:
from sklearn.metrics import accuracy_score,precision_score

accuracy = accuracy_score(y_test,y_pred)

In [ ]:
print(accuracy)

In [ ]:
precision = precision_score(y_test,y_pred)

In [ ]:
print(precision)

In [229]:
import joblib

model_data = {
    'weights':w,
    'bias':b
}

joblib.dump(model_data,'fraud_model.pkl')

['fraud_model.pkl']